**CSI 5139 Assignment 3**

This assignment explores transfer learning.

# Setup

In [5]:
import os
from shutil import copy

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

# VGG-16

In [4]:
datagen = ImageDataGenerator()

train_batchsize = 32 # 160 images per 68 classes
test_batchsize = 4 # 20 images per 68 classes

train_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/train/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=train_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/test/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=test_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

print("Train generator length: ", len(train_generator))
print("Test generator length: ", len(test_generator))
print("Target size: ", train_generator.target_size)
#print("Generator indices: ", train_generator.class_indices)
#print("Generator filenames:", train_generator.filenames[:10])

Found 10880 images belonging to 68 classes.
Found 1360 images belonging to 68 classes.
Train generator length:  340
Test generator length:  340
Target size:  (128, 128)


#### Model

In [25]:
def vgg16(freeze=False, remove=False):
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
    if freeze:
        # Freeze the layers
        for layer in vgg_conv.layers[7:]:
            layer.trainable = False
    
    if remove:
        # Remove the layers
        layers = vgg_conv.layers
        for i in range(8):
            layers.pop()
    
    # Check individual layers
    for layer in vgg_conv.layers:
        print(layer, layer.trainable)
        
    return vgg_conv

#### Run model

In [27]:
epochs = 1

vgg_conv = vgg16(remove=True)
vgg_conv.summary()

model = Sequential()
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(68, activation = 'softmax'))

# train the new layers with the weights of the existing VGG layers ﬁxe
model.layers[0].trainable = False

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

model.summary()

#rmsprop = RMSprop(lr=1e-3)
sgd = SGD(lr=1e-)

model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

hist_vgg16 = model.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1,)

# Save the model
model.save('./runs/vgg-16.h5')
del model  # deletes the existing model

hist_vgg16 = hist_vgg16.history

# save history
with open('./runs/vgg-16.hist', 'w', encoding="utf-8") as fout:
    pprint(hist_vgg16, fout)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000002AE3BD652B0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BD65978> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BD65B38> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002AE3BD65780> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BED7240> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BD797F0> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002AE3BD345F8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BD3DB38> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BD299E8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE3BD14BE0> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002AE3BCFF5F8> True
______________________________

KeyboardInterrupt: 